# Protein embeddings improve phage-host interaction prediction

**Mark Edward M. Gonzales<sup>1, 2</sup>, Jennifer C. Ureta<sup>1, 2</sup> & Anish M.S. Shrestha<sup>1, 2</sup>**

<sup>1</sup> Bioinformatics Laboratory, Advanced Research Institute for Informatics, Computing and Networking, De La Salle University, Manila, Philippines <br>
<sup>2</sup> Department of Software Technology, College of Computer Studies, De La Salle University, Manila, Philippines 

{mark_gonzales, jennifer.ureta, anish.shrestha}@dlsu.edu.ph

<hr>

## 💡 Dataset

**To download the dataset via INPHARED:**

1. Clone the latest version of INPHARED from its official [repository](http://github.com/RyanCook94/inphared). 
2. Download the PHROG HMMs (`all_phrogs.hmm`) from [Millard Lab](http://s3.climb.ac.uk/ADM_share/all_phrogs.hmm.gz) or from [Google Drive](https://drive.google.com/file/d/1oX-oOJzIynC6XKWWAvnd57CKqEV3t3kc/view?usp=sharing), and save it in the same folder as `inphared.pl`. 
3. Download the GenomesDB directory of INPHARED from this [link](https://millardlab-inphared.s3.climb.ac.uk/GenomesDB_20201412.tar.gz), and save it inside the same folder as `inphared.pl`.
4. Run the following script to execute the INPHARED pipeline (refer to the [documentation](https://github.com/RyanCook94/inphared#usage) for details): 
   - `perl inphared.pl -e exclusion_list.txt -P all_phrogs.hmm -c 0`
   - Notes:
      - The goal of running the pipeline is to complete populating the `GenomesDB` directory. Note that this may take some time since genome annotation is performed.
      - Note that running the INPHARED pipeline will load the most recent data. Our work uses the September data. For reproducibility, we have provided the following files to load the September data: 
        - `16Sep2022_data_excluding_refseq.tsv` (should have already been included when the repository was cloned) <br>
        - `16Sep2022_phages_downloaded_from_genbank.gb` (can be downloaded from [Google Drive](https://drive.google.com/file/d/14LG1iGa1CqPbAjofZT1EY8VKnE8Iy45Q/view?usp=sharing)) <br>
        
**After INPHARED finishes running:**

1. Save `16Sep2022_data_excluding_refseq.tsv` and `16Sep2022_phages_downloaded_from_genbank.gb` inside the `inphared` directory located in the same folder as this notebook. The folder structure should look like this:<br> <br>
   `experiments` (parent folder of this notebook) <br> 
     ↳ `inphared` <br>
     &nbsp; &nbsp; ↳ `16Sep2022_data_excluding_refseq.tsv` <br>
     &nbsp; &nbsp; ↳ `16Sep2022_phages_downloaded_from_genbank.gb`  <br>
     ↳ `1. Sequence Preprocessing.ipynb` (this notebook) <br> <br>
        
2. Create a directory named `datasets` in the **root** of this project.
   - Transfer `GenomesDB` to `datasets/inphared/inphared`.
   - The folder structure should look like this: <br> <br>
        `root` <br>
        ↳ `datasets` <br>
        &nbsp; &nbsp; ↳ `inphared` <br> 
        &nbsp; &nbsp;&nbsp; &nbsp; ↳ `inphared`  <br>
        &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; ↳ `GenomesDB`  <br>
        &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; ↳ `AB002632`  <br>
        &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; ↳ ... <br>
        ↳ `experiments` <br>
        &nbsp; &nbsp; ↳ `1. Sequence Preprocessing.ipynb` (this notebook)

<hr>

## 📁 Output Files
If you would like to skip running this notebook, you may download the output FASTA files from [Google Drive](https://drive.google.com/drive/folders/16ZBXZCpC0OmldtPPIy5sEBtS4EVohorT?usp=sharing). Save the results inside the `inphared` directory located in the same folder as this notebook. The folder structure should look like this:

`experiments` (parent folder of this notebook) <br> 
↳`inphared` <br>
&nbsp; &nbsp;↳ `fasta` <br>
&nbsp; &nbsp;&nbsp; &nbsp; ↳ `hypothetical` <br>
&nbsp; &nbsp;&nbsp; &nbsp; ↳ `nucleotide` <br>
&nbsp; &nbsp;&nbsp; &nbsp; ↳ `rbp` <br>
↳ `1. Sequence Preprocessing.ipynb` (this notebook) <br>

Intermediate output files (i.e., those saved in `temp/preprocessing`) should have already been included when the repository was cloned.

<hr>

# Part I: Preliminaries

Import the necessary libraries and modules.

In [1]:
import os
import shutil
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ConstantsUtil import ConstantsUtil
from SequenceParsingUtil import SequenceParsingUtil

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

pd.options.mode.chained_assignment = None

The parameter of `ConstantsUtil` corresponds to the download date of the dataset. 

In particular, when [INPHARED](https://github.com/RyanCook94/inphared) is run, among the files generated are:
- `<download_date>_data_excluding_refseq.tsv` <br>
- `<download_date>_phages_downloaded_from_genbank.gb` <br>

The download date of the dataset is `download_date`.

In [3]:
constants = ConstantsUtil('16Sep2022')
util = SequenceParsingUtil(constants.DISPLAY_PROGRESS, constants.MISSPELLING_THRESHOLD, constants.MIN_LEN_KEYWORD)

In [4]:
inphared_raw = pd.read_csv(f'{constants.INPHARED_TSV}', sep='\t')
inphared_raw = inphared_raw.drop_duplicates(ignore_index = True)
inphared_raw.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,LR756511,uncultured phage,uncultured phage environmental samples Viruses,595163,True,42.110,DNA,26-MAR-2020,1080,20.648148,79.351852,93.248236,NaN,62,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
1,LR756508,uncultured phage,uncultured phage environmental samples Viruses,484177,True,39.553,DNA,26-MAR-2020,683,98.389458,1.610542,89.636022,NaN,31,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
2,LR756504,uncultured phage,uncultured phage environmental samples Viruses,636363,True,26.393,DNA,26-MAR-2020,920,51.956522,48.043478,92.786193,NaN,34,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
3,LR756503,uncultured phage,uncultured phage environmental samples Viruses,735411,True,32.225,DNA,26-MAR-2020,1014,30.276134,69.723866,93.113239,NaN,56,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
4,LR756502,uncultured phage,uncultured phage environmental samples Viruses,642428,True,31.472,DNA,26-MAR-2020,971,54.582904,45.417096,94.525145,NaN,45,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified


In [5]:
inphared_raw.shape

(18389, 27)

<hr>

# Part II: Preprocessing of Host Information

For entries with `Unspecified` host in INPHARED, obtain the isolation host information from GenBank, if it makes sense.

In [6]:
util.set_inphared_gb(constants.INPHARED_GB)

In [7]:
inphared_unspec_host = inphared_raw.loc[inphared_raw['Host'] == 'Unspecified']
inphared_unspec_host.reset_index(inplace = True, drop = True)
inphared_unspec_host.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,LR756511,uncultured phage,uncultured phage environmental samples Viruses,595163,True,42.110,DNA,26-MAR-2020,1080,20.648148,79.351852,93.248236,NaN,62,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
1,LR756508,uncultured phage,uncultured phage environmental samples Viruses,484177,True,39.553,DNA,26-MAR-2020,683,98.389458,1.610542,89.636022,NaN,31,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
2,LR756504,uncultured phage,uncultured phage environmental samples Viruses,636363,True,26.393,DNA,26-MAR-2020,920,51.956522,48.043478,92.786193,NaN,34,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
3,LR756503,uncultured phage,uncultured phage environmental samples Viruses,735411,True,32.225,DNA,26-MAR-2020,1014,30.276134,69.723866,93.113239,NaN,56,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
4,LR756502,uncultured phage,uncultured phage environmental samples Viruses,642428,True,31.472,DNA,26-MAR-2020,971,54.582904,45.417096,94.525145,NaN,45,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified


In [8]:
phages_unspec_host = util.get_phages_unspec_host(inphared_unspec_host)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [9]:
unfiltered_hosts = util.get_unfiltered_hosts()
genus_typo = util.get_genus_typo(f'{constants.GENUS_TYPO}')
unfiltered_suspected_genera = util.get_unfiltered_suspected_genera(constants.CANDIDATE_REGEX)

excluded_hosts = util.get_excluded_hosts(constants.ARCHAEA, constants.BACTERIA_NOT_GENUS, constants.EXCLUDED_HOSTS)
valid_hosts = util.get_valid_hosts()

Add another layer of checking to identify typos in GenBank annotations, as well as oversights during manual filtering:
- Recheck genera in `valid_hosts` that are not in INPHARED's `Host` column.
- Identify entries with minimum edit distance &leq; 2 (likely misspellings).
<br>

Adjust the exclusion and typo correction files in the `preprocessing` directory accordingly, and rerun the previous cell, as needed.

In [10]:
inphared_hosts = set(inphared_raw['Host'].str.lower())
valid_hosts - inphared_hosts

{'silvanigrella'}

In [11]:
valid_hosts_list = list(valid_hosts)
for i in range(len(valid_hosts_list)):
    for j in range(i + 1, len(valid_hosts_list)):
        if util.is_possible_misspelling(valid_hosts_list[i], valid_hosts_list[j]):
            print(valid_hosts_list[i], valid_hosts_list[j])

microbacterium mycobacterium


Update `host` column based on the manually filtered info from GenBank.

In [12]:
inphared_augmented = inphared_raw.copy(deep = True)
util.update_host_column(constants.CANDIDATE_REGEX, inphared_unspec_host, inphared_augmented)

inphared_augmented['Host'] = inphared_augmented['Host'].str.lower()
inphared_augmented['Host'].value_counts()

Index: 424 | Name: ON970599 | Host: gordonia
Processed 1000 records
Processed 2000 records
Index: 2105 | Name: MN940411 | Host: staphylococcus
Processed 3000 records
Index: 3471 | Name: AF125163 | Host: vibrio
Index: 3477 | Name: AF063097 | Host: escherichia
Processed 4000 records
Index: 3576 | Name: OK499975 | Host: proteus
Index: 3705 | Name: OK649958 | Host: haloferax
Index: 3761 | Name: MW822148 | Host: mycobacterium
Processed 5000 records
Index: 4552 | Name: MH590600 | Host: microbacterium
Index: 4628 | Name: MH271321 | Host: microbacterium
Index: 4643 | Name: MH155873 | Host: microbacterium
Index: 5392 | Name: LC625742 | Host: enterococcus
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Index: 5662 | Name: MT588083 | Host: salmonella
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 1800

unspecified                                     2566
mycobacterium                                   2189
escherichia                                     1627
vibrio                                           852
pseudomonas                                      826
salmonella                                       744
streptococcus                                    730
klebsiella                                       628
gordonia                                         579
staphylococcus                                   508
enterobacteria                                   498
microbacterium                                   459
bacillus                                         377
lactococcus                                      364
synechococcus                                    362
arthrobacter                                     340
streptomyces                                     310
flavobacterium                                   247
acinetobacter                                 

### Polyvalent Phages

There are only few polyvalent (multi-host) phages in our database. Hence, for simplicity, we map each polyvalent phage only to its host with the highest number of interacting phages in the dataset.

In [13]:
multiple_hosts = set()

for host, count in dict(inphared_augmented['Host'].value_counts()).items():
    if '|' in host:
        multiple_hosts.add(host)
        print(host, ':', count)

escherichia | chryseobacterium | pseudomonas : 4
phormidium | plectonema : 1


In [14]:
for host in multiple_hosts:
    # We have confirmed that the first host is the host with the highest number of interacting phages in the dataset.
    inphared_augmented.loc[inphared_augmented['Host'] == host, 'Host'] = host.split(' | ')[0]

In [15]:
inphared = inphared_augmented.loc[inphared_augmented['Host'] != 'unspecified']
inphared.reset_index(inplace = True, drop = True)
inphared.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,MK250029,Prevotella phage Lak-C1,Prevotella phage Lak-C1 Myoviridae Caudovirice...,540217,True,25.796,DNA,13-JAN-2019,830,47.108434,52.891566,68.324951,NaN,30,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
1,MK250028,Prevotella phage Lak-B9,Prevotella phage Lak-B9 Myoviridae Caudovirice...,550053,True,26.012,DNA,13-JAN-2019,859,52.270081,47.729919,69.188424,NaN,29,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
2,MK250027,Prevotella phage Lak-B8,Prevotella phage Lak-B8 Myoviridae Caudovirice...,551627,True,26.022,DNA,13-JAN-2019,860,53.023256,46.976744,69.318761,NaN,33,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
3,MK250026,Prevotella phage Lak-B7,Prevotella phage Lak-B7 Myoviridae Caudovirice...,550702,True,26.020,DNA,13-JAN-2019,859,53.201397,46.798603,69.363285,NaN,33,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
4,MK250025,Prevotella phage Lak-B6,Prevotella phage Lak-B6 Myoviridae Caudovirice...,546689,True,26.029,DNA,13-JAN-2019,847,52.656434,47.343566,69.118274,NaN,30,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.


Replace the genera of some hosts with standard NCBI Taxonomy nomenclature.

In [16]:
inphared['Host'].replace(util.get_ncbi_standard_nomenclature(constants.NCBI_STANDARD_NOMENCLATURE), inplace = True)
inphared['Host'].value_counts()

mycobacterium                2189
escherichia                  1632
vibrio                        852
pseudomonas                   826
salmonella                    744
streptococcus                 730
klebsiella                    628
gordonia                      579
enterobacter                  558
staphylococcus                508
microbacterium                459
bacillus                      377
lactococcus                   364
synechococcus                 362
arthrobacter                  340
streptomyces                  310
flavobacterium                247
acinetobacter                 196
enterococcus                  185
rhizobium                     154
aeromonas                     140
erwinia                       137
shigella                      128
propionibacterium             125
yersinia                      120
rheinheimera                  110
campylobacter                  95
pectobacterium                 92
xanthomonas                    91
lactobacillus 

Just verifying that all the entries are unique

In [17]:
inphared.shape[0] == inphared['Accession'].nunique()

True

###  Some Statistics

Statistics on phages

In [18]:
print("Original INPHARED:\t\t", inphared_raw.shape[0])
print("With Specified Hosts:\t\t", inphared_raw.shape[0] - inphared_unspec_host.shape[0])
print("Plus Manually Filtered Hosts:\t", inphared.shape[0])

Original INPHARED:		 18389
With Specified Hosts:		 15739
Plus Manually Filtered Hosts:	 15823


Statistics on host genera

In [19]:
print("Num. of Unique Host Genera:\t", inphared['Host'].nunique())

Num. of Unique Host Genera:	 279


Save the entries with host information to a CSV file.

In [20]:
inphared.to_csv(os.path.join(constants.TEMP_PREPROCESSING, constants.INPHARED_WITH_HOSTS), index = False)

<hr>

# Part III: Selection of Annotated Receptor-Binding Proteins (RBPs)

Load only the phage entries with host data.

In [21]:
inphared = pd.read_csv(f'{constants.TEMP_PREPROCESSING}/{constants.INPHARED_WITH_HOSTS}')
util.set_inphared(inphared)

inphared.shape

(15823, 27)

Identify which phage entries have CDS information and which do not.

In [22]:
no_cds_annot = util.get_no_cds_annot()

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [23]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.NO_CDS_ANNOT),'wb') as no_cds_annot_file:
    pickle.dump(no_cds_annot, no_cds_annot_file)

In [24]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.NO_CDS_ANNOT}','rb') as no_cds_annot_file:
    no_cds_annot = pickle.load(no_cds_annot_file)
    
util.set_no_cds_annot(no_cds_annot)

### Some statistics

In [25]:
print("Without CDS info:\t", len(no_cds_annot))
print("With CDS info:\t\t", inphared.shape[0] - len(no_cds_annot))

Without CDS info:	 665
With CDS info:		 15158


## *A. Process entries with CDS information*

- Use regex from this [RBP prediction study](https://www.mdpi.com/1999-4915/14/6/1329) by Boeckaerts <i>et al.</i> (2022) to select the annotated RBPs
   - We modify this regex to:
     - Make more accommodating to typos (e.g., look for `bind` instead of `binding`)
     - Allow multiple spaces between tokens
   - For comparison:
     - Original regex: `tail.?(?:spike|fib(?:er|re))|^recept(?:o|e)r.?(?:binding|recognizing).*(?:protein)?|^RBP`
     - Modified regex: `tail?(.?|\s*)(?:spike?|fib(?:er|re))|recept(?:o|e)r(.?|\s*)(?:bind|recogn).*(?:protein)?|(?<!\w)RBP(?!a)`
     - Annotations captured by modified regex but not by old regex:
       - `receptor recognition protein`
   - The same [study](https://www.mdpi.com/1999-4915/14/6/1329) has an exclusion list that covers proteins related to RBPs but are not RBPs themselves (e.g., assembly and portal proteins).
       <br><br>

- If product does not match regex for RBP, check if it is a hypothetical protein
   - The same [study](https://www.mdpi.com/1999-4915/14/6/1329) has a list of keywords associated with hypothetical proteins
      - We remove `putative`, `probable`, and `probably` since they are indicative of a product with a putative (rather than an unknown) function.
   - Make more accommodating to typos by allowing keywords with minimum edit distance ≤ 2 (likely misspellings)
   - Create a manual list to handle cases where the keyword `hypothetical` is present but the putative function is already given, as in the case of:
       - `conserved hypothetical lipoprotein`
       - `hypothetical host-like ribonucleoside diphosphate reductase`       <br><br>
      
- Predict whether these hypothetical proteins are RBPs using the XGBoost model from the [RBP prediction study](https://www.mdpi.com/1999-4915/14/6/1329) by Boeckaerts <i>et al.</i> (2022).

In [26]:
util.set_token_delimiter(constants.TOKEN_DELIMITER)
util.construct_keyword_list(constants.HYPOTHETICAL_KEYWORDS, constants.RBP_RELATED_NOT_RBP, constants.PUTATIVE_FUNCTIONS)

In [27]:
annot_products = util.get_annot_products()

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [28]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.ANNOT_PRODUCTS),'wb') as annot_products_file:
    pickle.dump(annot_products, annot_products_file)

In [29]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.ANNOT_PRODUCTS}','rb') as annot_products_file:
    annot_products = pickle.load(annot_products_file)
    
util.set_annot_products(annot_products)
for annot_product in sorted(list(annot_products)):
    print(annot_product)

(2Fe-2S)-binding protein
(E)-4-hydroxy-3-methylbut-2-enyl-diphosphate synthase
(NAD(+)) DNA ligase
(R,S)-reticuline 7-O-methyltransferase-like isoform X1
(RecA-like) recombination and repair protein
(p)ppGpp synthase/hydrolase
(p)ppGpp synthetase, RelA/SpoT family
(pyro)phosphatase or 5'(3')-deoxyribonucleotidase
(thymidylate) synthase
0.2
0.3 protein
0.3B protein
0.4 protein
0.6
0.6A protein
0.6B protein
1
1, 4-beta-N-acetylmuramidase
1,4-alpha-glucan (glycogen) branching enzyme
1,4-alpha-glucan (glycogen) branching enzyme, GH-13-type
1,4-alpha-glucan (glycogen) branching enzyme, GH-13-type (EC
1,4-beta-N-acetylmuramidase
1,4-beta-N-acetylmuramidase or lysozyme
1,4-dihydroxy-6-naphthoate synthase
1,6-anhydro-N-acetylmuramyl-L-alanine amidase
1-(5-phosphoribosyl)-5-amino-4-imidazole- carboxylate carboxylase
1-(5-phosphoribosyl)-5-amino-4-imidazole- carboxylatecarboxylase
1-acyl-sn-glycerol-3-phosphate acyltransferase
1-aminocyclopropane-1-carboxylate deaminase
1-deoxy-D-xylulose-5-phos

Beta glucosyl transferase
Beta-lactamase SHV-2
BhlA
Bifunctional DNA primase/polymerase
Bifunctional DNA primase/polymerase, N-terminal
Bifunctional NMN adenylyltransferase/Nudix hydrolase
Big-1 domain-containing protein
Bis(5'-nucleosyl)-tetraphosphatase PrpE
Bla
BlaI/MecI/CopY family transcriptional regulator
BmpA
BmpB
BmpC
BofL
BofR
Bor
Bor lipoprotein
Bor protein
Bor protein precursor
BplA
BplB
BppU domain protein
BppU family baseplate upper protein
BppU family phage baseplate upper protein
BppU-like baseplate protein
BppU_N domain-containing protein
BrnA-like antitoxin
BrnT family toxin
BrnT-like antitoxin
BrnT-like toxin
Bro family antirepressor
Bro family toxin component
Bro-N domain protein
Bro-N family protein
Bro-N/ORF6C domain-containing antirepressor
BroN DNA binding protein
BspA family leucine rich repeat surface protein
BspA family leucine-rich repeat surface protein
BsuMI modification methylase subunit
BtrN protein
BtuR/CobO ATP:corrinoid adenosyltransferase
C
C (capsid 

D/B-b-hydroxylase DiO
D1
D10 protein
D108-specific protein
D11 protein
D11-like protein
D12 class N6 adenine-specific DNA methyltransferase
D12 class N6 adenine-specific DNA methyltransferase family protein
D12 class N6 adenine-specific DNAmethyltransferase
D13; putative exonuclease SbcCD, C subunit (ACLAME 108)
D14 Protein
D14 protein
D14 protein/putative Holiday junction resolvase (ACLAME 1108)
D14 protein/putative holiday junction resolvase
D14 protein/putative holiday junction resolvase (ACLAME 1108)
D14 protein/putative resolvase
D14-like protein
D152
D154
D2
D2 protein
D3 protein
D3-like protein
D383
D5 protein
D5 protein/DNA-binding protein
D5 transcription factor
D5-like protein
D6 protein
D60
DAN primase/helicase
DBP
DCM methylase
DCMP deaminase
DCTP deaminase
DCTP pyrophosphatase
DD-transpeptidase
DDE endonuclease
DDE superendonuclease family protein
DDE transposase
DDE-type integrase/transposase
DDE-type integrase/transposase/recombinase
DEAD box family helicase
DEAD box hel

DNA-directer RNA polymerase beta subunit
DNA-endonuclease-like protein
DNA-gyrase A-subunit
DNA-gyrase B-subunit
DNA-gyrase IV A-subunit
DNA-invertase
DNA-invertase hin
DNA-methylase
DNA-methyltransferase
DNA-methyltransferase subunit M
DNA-methyltransferase type II restriction modification system
DNA-packaging protein
DNA-packaging protein A
DNA-packaging protein B
DNA-packaging protein FI
DNA-packaging protein small subunit
DNA-packing protein small subunit
DNA-polymerase
DNA-polymerase catalytic subunit
DNA-primase
DNA-sulfur modification-associated
DNA-topoisomerase II B-subunit
DNA/RNA binding domain-containing protein
DNA/RNA binding protein
DNA/RNA endonuclease G
DNA/RNA helicase
DNA/RNA helicase of superfamily II
DNA/RNA helicase protein
DNA/RNA helicase superfamily II
DNA/RNA helicase-like protein
DNA/RNA non-specific endonuclease
DNA/RNA polymerase
DNA/RNA repair helicase
DNA/RNA-binding 3-helical bundle
DNA/RNA-binding protein
DNA/protein translocase
DNAB helicase
DNAB-like 

DenB DNA endonuclease IV
DenB-like DNA endonuclease IV
DenB-like endonuclease
DenB.1 hypothetical protein
DenV
DenV DNA endonuclease V
DenV Endonuclease V
DenV endonuclease
DenV endonuclease V
DenV endonuclease V N-glycosylase UV repair enzyme
DenV endonuclease V, N-glycosylase UV repair enzyme
DenV-like UV repair enzyme
DeoR family transcriptional regulator
DeoR/GlpR transcriptional regulator
Deoxyadenosine kinase (EC / Deoxyguanosine kinase (EC
Deoxyadenosine kinase DnK
Deoxyadenosine/Deoxyguanosine kinase
Deoxycytidine triphosphate deaminase
Deoxycytidine triphosphate deaminase (dUMP-forming)
Deoxycytidylate 5-hydroxymethyltransferase
Deoxycytidylate 5-hydroxymethyltransferase (EC
Deoxycytidylate deaminase
Deoxyguanosine kinase
Deoxynucleotide monophosphate kinase
Deoxynucleotide monophosphate kinase #T4-like phage gp1 #T4 GC1586
Deoxynucleotide monophosphate kinase (EC
Deoxynucleotide monophosphate kinase (EC #T4-like phage gp1 #T4 GC1586
Deoxyuridine 5'-triphosphate nucleotidohydr

...

putative replication factor C small subunit
putative replication factor C small subunit / DNA polymerase clamp loader subunit
putative replication function protein
putative replication gene B protein
putative replication helicase
putative replication initiation factor
putative replication initiation protein
putative replication initiation protein P12
putative replication initiator
putative replication initiator protein
putative replication initiator protein A
putative replication origin binding protein
putative replication origin-binding protein
putative replication protein
putative replication protein 18
putative replication protein DnaC
putative replication protein O
putative replication protein P
putative replication protein RepA
putative replication protein RepB
putative replication protein large subunit
putative replication protein p
putative replication termination protein
putative replication terminator protein
putative replication-associated protein
putative replicative DNA hel

putative transmembrane region domain containing protein
putative transpeptidase family protein
putative transport protein
putative transporter
putative transporter, ABC binding casette-like
putative transposase
putative transposase A
putative transposase A subunit
putative transposase B
putative transposase B subunit
putative transposase IS200-family protein
putative transposase ISCaje3 family
putative transposase ISCaje4 family
putative transposase OrfA protein of IS629
putative transposase OrfB protein of IS629
putative transposase domain-containing protein
putative transposase fusion protein
putative transposase like protein
putative transposase, IS607 family
putative transposase, ISCaje3 family
putative transposase, Tn7_Tnp_TnsA_N superfamily
putative transposase-like protein
putative transposase-like protein [Escherichiaphage vB_EcoM_PHB05]
putative transposon-related DNA-binding protein
putative transposon-related dna-binding protein
putative trimeric spike protein
putative tripa

ribonucleoside-diosate reductase subunit beta
ribonucleoside-diphosphatase reductase small subunit
ribonucleoside-diphosphate alpha subunit
ribonucleoside-diphosphate beta subunit
ribonucleoside-diphosphate reductase
ribonucleoside-diphosphate reductase (alpha)-like protein
ribonucleoside-diphosphate reductase 1 alpha chain
ribonucleoside-diphosphate reductase 1 alpha subunit
ribonucleoside-diphosphate reductase 1 subunit
ribonucleoside-diphosphate reductase 1 subunit alpha
ribonucleoside-diphosphate reductase 1 subunit beta
ribonucleoside-diphosphate reductase 1subunitalpha
ribonucleoside-diphosphate reductase 2 subunit alpha
ribonucleoside-diphosphate reductase 2 subunit beta
ribonucleoside-diphosphate reductase I subunit alpha
ribonucleoside-diphosphate reductase I subunit beta
ribonucleoside-diphosphate reductase R2
ribonucleoside-diphosphate reductase R2/beta subunit
ribonucleoside-diphosphate reductase alpha
ribonucleoside-diphosphate reductase alpha (large) subunit
ribonucleosid

sulfotransferase-like protein
sulfurtransferase
super infection exclusion protein
super infection immunity protein
super-infection exclusion protein
super-infection exclustion protein
superantigen A
superantigen-encoding pathogenicity island protein
superfamily I DNA and RNA helicase
superfamily I DNA or RNA helicase
superfamily I DNA/RNA helicase
superfamily II DNA or RNA helicase
superfamily II DNA/RNA helicase
superfamily II DNA/RNA helicase [Enterobacteria phage 9g]
superfamily II DNA/RNA helicase, SNF2 family
superfamily II helicase
superfamily II helicase restriction enzyme
superfamily II helicase/restriction enzyme
superfamily member TIGR01575
superfamily protein
superinfection exclusion
superinfection exclusion lipoprotein
superinfection exclusion protein
superinfection exclusion protein A
superinfection exclusion protein B
superinfection exclusion protein Cor-like protein
superinfection exclusion protein[Escherichiaphage PMBT57]
superinfection immunity protein
superoxide dismu

topoisomerase IA
topoisomerase IB
topoisomerase II
topoisomerase II domain-containing protein
topoisomerase II large subunit
topoisomerase II large subunit N-terminal region
topoisomerase II medium subunit
topoisomerase II medium subunit [Escherichia phage e11/2]
topoisomerase II medium subunit [Escherichia phage wV7]
topoisomerase II small subunit
topoisomerase II subunit
topoisomerase II, large subunit
topoisomerase II, large subunit, N-terminal region
topoisomerase II, medium subunit
topoisomerase IIM protein
topoisomerase IV subunit
topoisomerase IV subunit A
topoisomerase IV subunit B
topoisomerase IV subunit B protein
topoisomerase medium subunit
topoisomerase primase
topoisomerase subunit
topoisomerase-primase
topoisomerase-primase domain containing protein
topoisomerase-primase domain protein
topoisomerase-primase domain-containing protein
toprim domain containing protein
toprim domain protein
toprim domain-containing protein
toprim protein
toprim-domain containing protein
topr

In [30]:
rbp_products, hypothetical_proteins = util.get_rbp_hypothetical_proteins(constants.RBP_REGEX)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records
Processed 32000 records
Processed 33000 records
Processed 34000 records


In [31]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_PRODUCTS),'wb') as rbp_products_file:
    pickle.dump(rbp_products, rbp_products_file)
    
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.HYPOTHETICAL_PRODUCTS),'wb') as hypothetical_proteins_file:
    pickle.dump(hypothetical_proteins, hypothetical_proteins_file)

In [32]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.RBP_PRODUCTS}','rb') as rbp_products_file:
    rbp_products = pickle.load(rbp_products_file)
    
util.set_rbp_products(rbp_products)
rbp_products

{'adhesin tip for distal long tail fiber',
 'bacteriophage tail fiber protein',
 'central straight tail fiber',
 'central tail fiber',
 'central tail fiber j',
 'central tail fiber receptor binding protein',
 'central tail spike',
 'colanidase tailspike',
 'colanidase tailspike [enterobacteria phage ecgd1]',
 'conserved hypothetical tail fiber protein',
 'conserved tail fiber protein',
 'defective tail fiber protein',
 'depolymerase tail fiber protein',
 'distal long tail fiber adhesin',
 'distal long tail fiber subunit',
 'distal tail fiber protein',
 'distal/tail fiber protein',
 'duf1983 containing putative tail fiber protein',
 'endolysin tail fiber',
 'endolysin tail fiber hydrolase',
 'endosialidase tailspike',
 'endosialidase tailspike protein',
 'fhua receptor-binding tail protein',
 'fusion long tail fiber distal subunit',
 'gp12 short tail fiber protein',
 'gp12 short tail fibers',
 'gp12 short tail fibers protein',
 'gp21, tail fiber protein',
 'gp25, tail fiber',
 'gp36 put

In [33]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.HYPOTHETICAL_PRODUCTS}','rb') as hypothetical_proteins_file:
    hypothetical_proteins = pickle.load(hypothetical_proteins_file)
    
util.set_hypothetical_proteins(hypothetical_proteins)
for protein in hypothetical_proteins:
    print(protein)

gp30.2 conserved hypothetical protein
protein of unknown function duf1493
rb69orf033c hypothetical protein
arn.3 hypothetical protein
hypothetical-protein | belonging to t4-like gc: 798
gp30.3 conserved hypothetical protein
hypothetical-protein | belonging to t4-like gc: 313
hypothetical helix-turn-helix domain protein
hypothetical protein containing coiled-coil segments
protein of unknown function duf1523
hypothetical protein_gp147
hypothetical protein [escherichia phage olb145]
cd.3 hypothetical protein
hypothetical protein_gp019
hypothetical protein [escherichia phagevb_ecom_phb05
hypothetical protein rdjlphi1_gp14
hypothetical protein jb75_0167 [escherichia phage vb_ecom_jb75]
hypothetical protein vn4_19 [vibrio phage n4]
nrdc.2 hypothetical protein
hyphothetical protein
hypothetical protein a368
gp44, hypothetical protein
hypothetical protein_gp031
protein of unknown function duf2163
alt.1 conserved hypothetical protein
hypothetical protein, duf3310
hypothetical protein g53_00272 

rb69orf053c hypothetical protein
vs.4 hypothetical protein
uncharacterized phage protein
protein of unknown function duf1360
hypothetical-protein | belonging to t4-like gc: 751
gp30.7 hypothetical protein
gp61.4 conserved hypothetical protein
gp55.2 hypothetical protein
cog1683: uncharacterized conserved protein / fig143828: hypothetical protein ybga
protein of unknown function duf4060
gp33, hypothetical protein
gp30.5 hypothetical protein
hypothetical protein_gp024
hypothetical protein_gp044
hypothetical protein_gp137
gp39.1 conserved hypothetical protein
protein of unknown function duf2774
dda.1 hypothetical protein
fig00639352: hypothetical protein
protein of unknown function duf1408
pset.2 hypothetical protein
gp46.2 conserved hypothetical protein
rb32orf217c hypothetical protein
hypothetical bacteriophage protein
hypothetical protein mrh.1
rb69orf123c hypothetical protein
hypothetical protein_gp177
hypothetical-protein | belonging to t4-like gc: 749
hypothetical protein, ninb homo

### Get distribution of RBP lengths

This preliminary analysis of the distribution of the RBP lengths covers both the sequences with CDS information and those where the CDS information features are predicted via Prokka. We carry it now in order since we need the lower and upper bounds for RBP lengths to generate the FASTA files.

In [34]:
len_distribution = util.generate_rbp_len_distribution()
util.generate_rbp_len_distribution_prokka(len_distribution, constants.INPHARED_GENOME)

lengths = []
for length, freq in len_distribution.items():
    for _ in range(freq):
        lengths.append(length)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [35]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_LENGTHS),'wb') as rbp_lengths_file:
    pickle.dump(lengths, rbp_lengths_file)

Statistically compute the lower and upper bounds for RBP lengths.

In [36]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.RBP_LENGTHS}','rb') as rbp_lengths_file:
    lengths = pickle.load(rbp_lengths_file)

Q1 = np.percentile(lengths, 25, interpolation = 'midpoint')
Q3 = np.percentile(lengths, 75, interpolation = 'midpoint')
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)

Lower bound: -533.0
Upper bound: 1587.0


### Generate FASTA for entries with CDS information

Generate FASTA files containing protein sequences of hypothetical proteins and RBPs

In [37]:
RBP_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.RBP}/{constants.GENBANK}'
HYPOTHETICAL_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.HYPOTHETICAL}/{constants.GENBANK}'

if not os.path.exists(RBP_NEW_DIR):
    os.makedirs(RBP_NEW_DIR)
    
if not os.path.exists(HYPOTHETICAL_NEW_DIR):
    os.makedirs(HYPOTHETICAL_NEW_DIR)

In [38]:
util.generate_rbp_hypothetical_fasta(RBP_NEW_DIR, HYPOTHETICAL_NEW_DIR, constants.LOWER_BOUND_RBP_LENGTH, 
                                     constants.UPPER_BOUND_RBP_LENGTH)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


The FASTA files are stored in the `inphared/fasta` directory, specifically in the `genbank` subdirectories.

## *B. Process entries without CDS information*

- Annotate genes using [Prokka](https://academic.oup.com/bioinformatics/article/30/14/2068/2390517), which calls [Prodigal](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-11-119) to predict coordinates of ORFs.
- Configure Prokka to perform functional annotation using [PHROGs](https://academic.oup.com/nargab/article/3/3/lqab067/6342220).
- Process in the same manner as the entries with CDS information.

In [39]:
RBP_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.RBP}/{constants.PROKKA}'
HYPOTHETICAL_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.HYPOTHETICAL}/{constants.PROKKA}'

if not os.path.exists(RBP_NEW_DIR):
    os.mkdir(RBP_NEW_DIR)
    
if not os.path.exists(HYPOTHETICAL_NEW_DIR):
    os.mkdir(HYPOTHETICAL_NEW_DIR)

Analyze the gene product annotations returned by PHROG.

In [40]:
annot_products_prokka = util.get_annot_products_prokka(constants.INPHARED_GENOME)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records


In [41]:
x, y = util.get_rbp_hypothetical_proteins_prokka(constants.RBP_REGEX, annot_products_prokka)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records


Combine the keywords for RBPs and hypothetical proteins from GenBank (i.e., entries with CDS information) and PHROG annotations (i.e., entries without CDS information).

In [42]:
util.set_rbp_products(rbp_products.union(x))
util.set_hypothetical_proteins(hypothetical_proteins.union(y))

In [43]:
rbp_products = rbp_products.union(x)
hypothetical_proteins = hypothetical_proteins.union(y)

In [44]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_PRODUCTS),'wb') as rbp_products_file:
    pickle.dump(rbp_products, rbp_products_file)
    
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.HYPOTHETICAL_PRODUCTS),'wb') as hypothetical_proteins_file:
    pickle.dump(hypothetical_proteins, hypothetical_proteins_file)

### Generate FASTA for entries without CDS information

Generate FASTA files containing protein sequences of hypothetical proteins and RBPs

In [45]:
util.generate_rbp_hypothetical_fasta_prokka(constants.INPHARED_GENOME, RBP_NEW_DIR, HYPOTHETICAL_NEW_DIR, 
                                            lower_bound, upper_bound)

The FASTA files are stored in the `inphared/fasta` directory, specifically in the `prokka` subdirectories.

### C. *Generate FFN containing the genomes of RBPs and hypothetical proteins*

In [46]:
if not os.path.exists(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.GENBANK}'):
    os.makedirs(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.GENBANK}')
    
if not os.path.exists(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.GENBANK}'):
    os.makedirs(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.GENBANK}')

In [47]:
util.generate_rbp_hypothetical_nucleotide(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.GENBANK}',
                                          f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.GENBANK}', 
                                          lower_bound, upper_bound)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


The FFN files are stored in the `inphared/fasta/nucleotide` directory, specifically in the `genbank` subdirectories.

In [48]:
if not os.path.exists(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.PROKKA}'):
    os.makedirs(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.PROKKA}')
    
if not os.path.exists(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.PROKKA}'):
    os.makedirs(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.PROKKA}')

In [49]:
util.generate_rbp_hypothetical_nucleotide_prokka(constants.INPHARED_GENOME,
                                                 f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.PROKKA}',
                                                 f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.PROKKA}', 
                                                 lower_bound, upper_bound)

The FFN files are stored in the `inphared/fasta/nucleotide` directory, specifically in the `prokka` subdirectories.